In [3]:
import shapely as shp
import requests
import pandas as pd
import geopandas as gpd
import numpy as np
import numpy as np

In [4]:
sportsfield_centers_gdf = gpd.read_file("data/sportsfield_centers_gdf", bbox=None, mask=None, rows=None, engine=None)
print()
coords = sportsfield_centers_gdf['geometry']
coords = shp.get_coordinates(coords)
coords = coords.tolist()
coord= coords[0]

In [23]:
# These are the variables that are included in the URL that we use.
# Other options are also possible, see open-meteo.com for more data.
variables = ["temperature_2m_max", "temperature_2m_min", "rain_sum", "snowfall_sum", "windspeed_10m_max", "winddirection_10m_dominant"]

# Create empty lists to store the means for each variable

urllat = coord[0]
urllon = coord[1]

# Set URL for different datatypes at each location
# open_meteo_url = f"http://archive-api.open-meteo.com/v1/archive?latitude={urllat}&longitude={urllon}&start_date=2022-01-01&end_date=2023-01-01&daily={','.join(variables)}&timezone=Africa/Cairo"
open_meteo_url = f"http://archive-api.open-meteo.com/v1/archive?latitude={urllat}&longitude={urllon}&start_date=2018-01-01&end_date=2023-01-01&daily={','.join(variables)}&timezone=Africa%2FCairo" 
# Request data from the URL
response = requests.get(open_meteo_url)
data = response.json()

    
weather_values = {var: [] for var in variables}
# weather_values['lat'] = []  # List to store latitude values
# weather_values['lon'] = []  # List to store longitude values
weather_values['max_temp'] = []
weather_values['min_temp'] = []

# weather_values['lat'].append(coord[0])
# weather_values['lon'].append(coord[1])

for var in variables:
    daily_data = data['daily']
    if var in daily_data:
        daily_var = daily_data[var]
        mean_value = np.mean(daily_var)
        weather_values[var].append(mean_value)


 21    130
 20    127
 22    122
 6     114
 8     108
 18    103
 7      95
 9      92
 5      90
 19     85
 17     82
 4      78
 14     66
 10     63
 11     62
 13     62
 16     59
 3      59
 15     52
 12     51
 1      31
 2      31
 23     26
 0      24
-1       9
 24      3
-3       2
-2       1
dtype: int64


In [27]:
def find_temperature_statistics(data, upper_bound = True):
    
    # If the temperature occurs more than 3 times per year on average, it is considered not tolerable as an extreme temperature (this is an estimate).  
    day_counts = len(data)
    min_occurrences = day_counts*3/365

    # Count the occurrences of each temperature
    temperatures_int = [int(x) for x in data]
    temperature_counts_around_range = pd.Series(temperatures_int).value_counts()
    # print(temperature_counts_around_range)


    # Filter temperatures that occur more than a certain number of days around the specified range
    filtered_temperatures_around_range = temperature_counts_around_range[temperature_counts_around_range > min_occurrences]

    if upper_bound == True: 
        return filtered_temperatures_around_range.index.max()
    else: 
        return filtered_temperatures_around_range.index.min()
            
# max_temp_data = data['daily']["temperature_2m_max"]    
# weather_values['max_temp'].append(find_temperature_statistics(max_temp_data, True))

min_temp_data = data['daily']["temperature_2m_min"]
print(find_temperature_statistics(min_temp_data, False))
# print(weather_values)
# Create a DataFrame to store the data
# df = pd.DataFrame(weather_values)



0


In [10]:
# #  These are the variables that are included in the URL that we use.
# # Other options are also possible, see open-meteo.com for more data.
# variables = ["temperature_2m_max", "temperature_2m_min", "rain_sum", "snowfall_sum", "windspeed_10m_max", "winddirection_10m_dominant"]

# # Create empty lists to store the means for each variable
# weather_values = {var: [] for var in variables}
# weather_values['lat'] = []  # List to store latitude values
# weather_values['lon'] = []  # List to store longitude values
# weather_values['max_temp'] = []
# weather_values['min_temp'] = []
# variables = ["temperature_2m_max", "temperature_2m_min", "rain_sum", "snowfall_sum", "windspeed_10m_max", "winddirection_10m_dominant"]

# coord=coords[0]

# urllat = coord[0]
# urllon = coord[1]

# weather_values['lat'].append(urllat)
# weather_values['lon'].append(urllon)

# # Set URL for different datatypes at each location
# # open_meteo_url = f"http://archive-api.open-meteo.com/v1/archive?latitude={urllat}&longitude={urllon}&start_date=2022-01-01&end_date=2023-01-01&daily={','.join(variables)}&timezone=Africa/Cairo"
# open_meteo_url = f"http://archive-api.open-meteo.com/v1/archive?latitude={urllat}&longitude={urllon}&start_date=2018-01-01&end_date=2023-01-01&daily={','.join(variables)}&timezone=Africa%2FCairo" 
# # Request data from the URL
# response = requests.get(open_meteo_url)
# data = response.json()

# for var in variables:
#     daily_data = data['daily']
#     if var in daily_data:
#         daily_var = daily_data[var]
#         mean_value = np.mean(daily_var)
#         print(mean_value)
#         weather_values[var].append(mean_value)



# def find_temperature_statistics(data, target_temperature, min_occurrences):
#     # Define the range around the target temperature (e.g., +/- 5°C)
#     range_values = list(range(target_temperature - 5, target_temperature + 6))

#     # Filter temperatures around the specified range
#     temperatures_around_range = data['Temperature_2m(°C)'].apply(lambda x: min(range_values, key=lambda y: abs(x - y)))

#     # Count the occurrences of each temperature around the specified range
#     temperature_counts_around_range = temperatures_around_range.value_counts()

#     # Filter temperatures that occur more than a certain number of days around the specified range
#     filtered_temperatures_around_range = temperature_counts_around_range[temperature_counts_around_range > min_occurrences]

#     # Find the minimum or maximum temperature around the specified range that occurred more than a certain number of times
#     if filtered_temperatures_around_range.empty:
#         return None
#     return filtered_temperatures_around_range.index.max()

# # Use the function for 40°C range (e.g., +/- 5°C)
# temp_data = data['daily']["temperature_2m_max"]

# max_temp_more_than_60_around_40 = find_temperature_statistics(temp_data, 40, 60)

# # Use the function for 0°C range (e.g., +/- 5°C)
# # min_temp_more_than_60_around_0 = find_temperature_statistics(Temp_2m, 0, 60)

# var = "temperature_2m_max"
# max_temperature = max(data['daily'][var])
# weather_values['max_temp'].append(max_temperature)

# var = "temperature_2m_min"
# min_temperature = min(data['daily'][var])
# weather_values['min_temp'].append(min_temperature)

# # Create a DataFrame to store the data
# df = pd.DataFrame(weather_values)

# # Create a GeoDataFrame from the DataFrame
# geometry = gpd.points_from_xy(df.lon, df.lat)
# gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')

22.437547892720307
13.008648056923919
1.7284619594964423
0.0007662835249042146
13.428680897646416
195.8927203065134


TypeError: list indices must be integers or slices, not str

In [ ]:
"""snippet for visualizing a test POLYGON"""

# df = pd.DataFrame(
#     {   "Latitude": [36.200724, 36.198916, 36.198567, 36.199512, 36.200563],
#         "Longitude": [36.131536, 36.129690, 36.132002, 36.133424, 36.132218],

#     }
# )

# potential_shelterspot = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude), crs="EPSG:4326")
# shape = potential_shelterspot.dissolve().convex_hull
# shapegdf = gpd.GeoDataFrame(geometry=shape)
# print(shapegdf)
# print(potential_shelterspot)
# center = shape.centroid
# centergdf = gpd.GeoDataFrame(geometry=center)
# print(centergdf)
# print(type(centergdf))
# coords = centergdf.get_coordinates()
# potential_shelterspot = potential_shelterspot.apppend(centergdf)